In [1]:
import numpy as np
import cv2

img = cv2.imread('testdata/1.png')
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
guidance = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

sigma_r = 0.1
sigma_s = 3
self_sigma_r = sigma_r
self_sigma_s = sigma_s
self_wndw_size = 6*sigma_s+1
self_pad_w = 3*sigma_s
    
BORDER_TYPE = cv2.BORDER_REFLECT
padded_img = cv2.copyMakeBorder(img, self_pad_w, self_pad_w, self_pad_w, self_pad_w, BORDER_TYPE)
padded_guidance = cv2.copyMakeBorder(guidance, self_pad_w, self_pad_w, self_pad_w, self_pad_w, BORDER_TYPE)
print(padded_guidance.shape)

### TODO ###
# Step 1: Calculus Gs -- constant matrix
gs = np.zeros((self_wndw_size, self_wndw_size))
for i in range(self_wndw_size):
    for j in range(self_wndw_size):
        gs[i,j] = np.exp(((i-self_pad_w)*(i-self_pad_w)+(j-self_pad_w)*(j-self_pad_w))/-(2*sigma_s*sigma_s))
print(gs.shape)
Gs = np.repeat(gs[np.newaxis,np.newaxis,:,:], guidance.shape[0], axis=0)
Gs = np.repeat(Gs, guidance.shape[1], axis=1)
print(Gs.shape)

# Step 2: Normalize guidance(gray)
padded_guidance = padded_guidance.astype('float64')
padded_img = padded_img.astype('float64')
padded_guidance = (padded_guidance-padded_guidance.min())/(padded_guidance.max()-padded_guidance.min())

# Step 3: Calculus Gr with guidance(gray), G:=Gr*Gs, JB-filter I'
output = np.zeros(img_rgb.shape)

for i in range(self_pad_w, padded_guidance.shape[0]-self_pad_w):
    for j in range(self_pad_w, padded_guidance.shape[1]-self_pad_w):
        Tp=padded_guidance[i,j]
        Tq=padded_guidance[i-self_pad_w:i+self_pad_w+1, j-self_pad_w:j+self_pad_w+1]   
        pw = np.divide(np.square(Tp-Tq), -2*self_sigma_r*self_sigma_r)
        print(pw.shape)
        Gr=np.exp(pw)
        G =np.multiply(Gs[i-self_pad_w, j-self_pad_w], Gr)
        W =g.sum(axis=1).sum(axis=0)
        Iq=padded_img[i-self_pad_w:i+self_pad_w+1, j-self_pad_w:j+self_pad_w+1,:]
        
        for k in range(img_rgb.shape[2]):
            output[i-self_pad_w, j-self_pad_w, k] = np.multiply(g,Iq[:,:,k]).sum(axis=1).sum(axis=0)/W 

print(output.shape)
#return np.clip(output, 0, 255).astype(np.uint8)

cv2.waitKey(0)
cv2.destroyAllWindows()

(318, 418)
(19, 19)
(300, 400, 19, 19)
(19, 19)


NameError: name 'g' is not defined